## Raw to csv preprocessing

In [ ]:
from utility.preprocessing2 import clean_data
DATA_SIZE=10000
BASE_PATH=f'data/{DATA_SIZE}'
clean_data(BASE_PATH)

### Feature engineering 

In [ ]:
from utility.preprocessing2 import clean_data,lower_case, preprocessing, upsample
from sklearn.model_selection import train_test_split
import pandas as pd
import os
DATA_SIZE=100000
df=pd.read_csv(f"data/process/final_{DATA_SIZE}.csv")

In [ ]:
# check data len 
DAYS=30
print(f"total Admission : {len(df)}")
print(f"admit within {DAYS} : {len(df[df['days']<=DAYS])} ")


In [ ]:
all_feature=['PatientGender', 'PatientRace','Age',
'PatientMaritalStatus', 'PatientLanguage',
'PatientPopulationPercentageBelowPoverty', 'AdmissionID',
'PrimaryDiagnosisCode', 'PrimaryDiagnosisDescription', 'Target', ]

### process_df and train_df we can use for evaluation

In [ ]:
#convert df in lower case
df=lower_case(df)
process_df=preprocessing(df,DAYS,all_feature) # whole dataset you will for evaluation
train_df=upsample(process_df,10000) #upsample low class

In [ ]:
train_df['Target'].value_counts()

### Train model and save

In [ ]:
from model.model2 import train_model
model,test,train = train_model(train_df)

### Evaluation 

In [ ]:
from testing.evaluation2 import evaluation_data
# use train_df or process_df  set True to get csv for manual comparison 
df_test=evaluation_data(train_df.copy(),'save_model/model.pkl',True)

In [ ]:
# df_test=evaluation_data(train_df,'save_model/model.pkl')
print(f"total={len(df_test)}\n")
print(df_test.Target.value_counts())
print(f"\n0 <-> 0 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==0)])}")
print(f"1 <-> 1 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==1)])}")
print(f"0 <-> 1 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==1)])}")
print(f"1 <-> 0 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==0)])}")

### inferences 

In [ ]:
import pandas as pd
data={'PatientGender':['male','female'],
      'PatientRace':['white','white'],
      'Age':[41,32],
      'PatientMaritalStatus':['single','single'],
      'PatientLanguage':['english','english'],
      'PatientPopulationPercentageBelowPoverty':[11.53,12.3],
      'AdmissionID':[2,1],
      'PrimaryDiagnosisCode':['t70.3','t70.3'],
      'PrimaryDiagnosisDescription':['Caisson disease [decompression sickness]','Caisson']
     }
test_df=pd.DataFrame(data)

In [ ]:
from testing.evaluation2 import infer
infer(pd.DataFrame(test_df))